In [6]:
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst
from sensor_msgs.msg import Image
from tqdm import tqdm
import argparse
import cv2
import numpy as np
import os
import re
import rosbag
import yaml

In [9]:
# Declare and initialize global variables.
anticipated_frame_duration = 0
rel_time = 0

def load_config_data(fl: str) -> dict:
    with open(fl) as f:
        cfg: dict = yaml.safe_load(f)
    return cfg

def print_dict(d: dict):
    for key, value in d.items():
        print(f'{key}: {value}')

def gstreamer_pipeline(loc):
    return (
        '''
        filesrc location={} !
        decodebin !
        nvvideoconvert !
        video/x-raw,format=RGBA !
        fakesink name=s
        '''.format(loc)
    )

def get_sec_and_nsec_parts_from_filename(fn):
    '''
    This function assumes that fn has the timestamp embedded in a specific pattern.
    The timestamp is represented as two integer numbers connected by an underscore ("_")
    The first integer has 10 digits and the second have 9 digits.

    The return values are the two parts of digits in their raw string format.
    '''
    m = re.search(r'_(\d{10}_\d+)$', fn)
    assert m is not None, f'{fn} does not contain the proper timestamp sub-string. '
    return m[1].split('_')

def on_frame_probe(pad, info):
    buf = info.get_buffer()

    frame = buffer_to_image(buf, pad.get_current_caps())

    # For extracting frames at a specific frame rate
    if not('prev_frame_time' in globals()):
        global prev_frame_time
        prev_frame_time = 0

    # For extracting frames between a specific time interval
    global rel_time, anticipated_frame_duration
    rel_time = buf.pts
    if cfg['start'] <= rel_time*1e-9 < cfg['stop'] or not cfg['extract_specific_time']:
        current_time = start_time + rel_time
        if ( current_time - prev_frame_time ) * 1e-9 >= anticipated_frame_duration:
            frame_name = str(current_time)
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            frame = cv2.resize(frame, (cfg['width'], cfg['height']))
            if cfg['rotate_img']:
                frame = cv2.rotate(frame, cv2.ROTATE_180)
            cv2.imwrite(os.path.join(cfg['save_dir'], frame_name+'.png'), frame)
            prev_frame_time = current_time

    return Gst.PadProbeReturn.OK

def buffer_to_image(buf, caps):
    caps_structure = caps.get_structure(0)
    height, width = caps_structure.get_value('height'), caps_structure.get_value('width')

    is_mapped, map_info = buf.map(Gst.MapFlags.READ)
    if is_mapped:
        try:
            image_array = np.ndarray(
                (height, width, pixel_bytes),
                dtype=np.uint8,
                buffer=map_info.data
            ).copy() # extend array lifetime beyond subsequent unmap
            return image_array[:,:,:3] # RGBA -> RGB
        finally:
            buf.unmap(map_info)

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument("config", type=str, default="")

args = parser.parse_args()

usage: ipykernel_launcher.py [-h] config
ipykernel_launcher.py: error: the following arguments are required: config


SystemExit: 2

In [10]:
cfg_path="./config/wildfire/subcanopy/extract.yaml"

cfg = load_config_data(cfg_path)

print('Loaded config:')
print_dict(cfg)

# Populate anticipated_frame_duration from cfg.
anticipated_frame_duration = 1 / cfg['frame_rate'] \
                             if cfg['specific_frame_rate'] \
                             else 0

os.makedirs(cfg['save_dir'], exist_ok=True)

filename = os.path.basename(cfg['video_path'])
filename = os.path.splitext(filename)[0]

# Try to find the timestamp imbedded in the filename.
sec_str, nsec_str = get_sec_and_nsec_parts_from_filename(filename)

sec = int(sec_str)
nano_sec = int(nsec_str)
start_time = sec*int(1e9) + nano_sec

pixel_bytes = 4
pipeline_str = gstreamer_pipeline(cfg['video_path'])

Gst.init(None)
pipeline = Gst.parse_launch(pipeline_str)

pipeline.get_by_name('s').get_static_pad('sink').add_probe(
    Gst.PadProbeType.BUFFER,
    on_frame_probe
)

print('Extracting Video ...')
pipeline.set_state(Gst.State.PLAYING)
try:
    while True:
        msg = pipeline.get_bus().timed_pop_filtered(
            Gst.SECOND,
            Gst.MessageType.EOS | Gst.MessageType.ERROR
        )

        # Show something such that the user knows it is doing its work.
        print('.', end='', flush=True)

        if rel_time*1e-9 > cfg['stop'] and cfg['extract_specific_time']:
            print('\nFinished extracting video until {} seconds'.format(cfg['stop']))
            break

        if msg:
            text = msg.get_structure().to_string() if msg.get_structure() else ''
            msg_type = Gst.message_type_get_name(msg.type)
            print(f'{msg.src.name}: [{msg_type}] {text}')
            break
finally:
    pipeline.set_state(Gst.State.NULL)
    print('')
    print('Extraction Complete')


Loaded config:
video_path: /root/data/wildfire/subcanopy/2023-04-27_FIRE-SGL-228/1/camera_0_1677761985_629121776.mp4
save_dir: /root/data/wildfire/subcanopy/2023-04-27_FIRE-SGL-228/1/camera_0
height: 514
width: 612
extract_specific_time: True
start: 2
stop: 3
specific_frame_rate: False
frame_rate: 5
rotate_img: False
Extracting Video ...
.
Finished extracting video until 3 seconds

Extraction Complete
